**Sistema de Gestión de Tienda**

Clases:

    Producto
    Cliente
    Orden
    ItemOrden
    Categoria
    Tienda

Relaciones:

    Un Producto pertenece a una Categoria.
    Un Cliente puede realizar múltiples Ordenes.
    Una Orden contiene múltiples ItemOrden.
    Cada ItemOrden está asociado a un Producto.
    La Tienda administra Productos, Clientes, Ordenes y Categorias.

Atributos y Métodos:

    Producto:
        Atributos: nombre, precio, categoria
        Métodos: mostrar_info()
    Cliente:
        Atributos: nombre, apellido, id_cliente
        Métodos: mostrar_info()
    Orden:
        Atributos: cliente, items, total
        Métodos: agregar_item(), calcular_total()
    ItemOrden:
        Atributos: producto, cantidad
        Métodos: calcular_subtotal()
    Categoria:
        Atributos: nombre
        Métodos: mostrar_info()
    Tienda:
        Atributos: productos, clientes, ordenes, categorias
        Métodos: registrar_producto(), registrar_cliente(), crear_orden(), mostrar_productos()

In [3]:
import tkinter as tk
from tkinter import ttk, messagebox

class Categoria:
    def __init__(self, nombre):
        self.nombre = nombre

    def mostrar_info(self):
        return f"Categoría: {self.nombre}"

class Producto:
    def __init__(self, nombre, precio, categoria):
        self.nombre = nombre
        self.precio = precio
        self.categoria = categoria

    def mostrar_info(self):
        return f"Producto: {self.nombre}, Precio: {self.precio}, Categoría: {self.categoria.nombre}"

class Cliente:
    def __init__(self, nombre, apellido, id_cliente):
        self.nombre = nombre
        self.apellido = apellido
        self.id_cliente = id_cliente

    def mostrar_info(self):
        return f"Cliente: {self.nombre} {self.apellido}, ID: {self.id_cliente}"

class ItemOrden:
    def __init__(self, producto, cantidad):
        self.producto = producto
        self.cantidad = cantidad

    def calcular_subtotal(self):
        return self.producto.precio * self.cantidad

    def mostrar_info(self):
        return f"{self.producto.nombre} x{self.cantidad} - Subtotal: ${self.calcular_subtotal():.2f}"

class Orden:
    def __init__(self, cliente):
        self.cliente = cliente
        self.items = []
        self.total = 0.0

    def agregar_item(self, item):
        self.items.append(item)
        self.calcular_total()

    def calcular_total(self):
        self.total = sum(item.calcular_subtotal() for item in self.items)
        return self.total

    def mostrar_info(self):
        items_info = "\n".join([item.mostrar_info() for item in self.items])
        return f"Orden para {self.cliente.nombre} {self.cliente.apellido}:\n{items_info}\nTotal: ${self.total:.2f}"

class Tienda:
    def __init__(self):
        self.productos = []
        self.clientes = []
        self.ordenes = []
        self.categorias = []

    def registrar_producto(self, producto):
        self.productos.append(producto)

    def registrar_cliente(self, cliente):
        self.clientes.append(cliente)

    def crear_orden(self, cliente):
        orden = Orden(cliente)
        self.ordenes.append(orden)
        return orden

    def mostrar_productos(self):
        return [producto.mostrar_info() for producto in self.productos]

    def obtener_ordenes_cliente(self, id_cliente):
        return [orden for orden in self.ordenes if orden.cliente.id_cliente == id_cliente]

# Funciones para la interfaz gráfica

def mostrar_productos():
    for widget in frame_productos.winfo_children():
        widget.destroy()
    productos = tienda.mostrar_productos()
    for producto in productos:
        label = tk.Label(frame_productos, text=producto, bg="lightblue", font=("Arial", 12))
        label.pack(pady=5)

def registrar_producto():
    nombre = entry_producto_nombre.get()
    precio = float(entry_producto_precio.get())
    categoria_nombre = entry_producto_categoria.get()
    
    categoria = next((c for c in tienda.categorias if c.nombre == categoria_nombre), None)
    if categoria is None:
        categoria = Categoria(categoria_nombre)
        tienda.categorias.append(categoria)
    
    producto = Producto(nombre, precio, categoria)
    tienda.registrar_producto(producto)
    messagebox.showinfo("Registro de Producto", "Producto registrado exitosamente")
    entry_producto_nombre.delete(0, tk.END)
    entry_producto_precio.delete(0, tk.END)
    entry_producto_categoria.delete(0, tk.END)

def registrar_cliente():
    nombre = entry_cliente_nombre.get()
    apellido = entry_cliente_apellido.get()
    id_cliente = int(entry_cliente_id.get())
    cliente = Cliente(nombre, apellido, id_cliente)
    tienda.registrar_cliente(cliente)
    messagebox.showinfo("Registro de Cliente", "Cliente registrado exitosamente")
    entry_cliente_nombre.delete(0, tk.END)
    entry_cliente_apellido.delete(0, tk.END)
    entry_cliente_id.delete(0, tk.END)

def crear_orden():
    id_cliente = int(entry_orden_cliente_id.get())
    cliente = next((c for c in tienda.clientes if c.id_cliente == id_cliente), None)
    if cliente is None:
        messagebox.showerror("Error", "Cliente no encontrado")
        return
    orden = tienda.crear_orden(cliente)
    messagebox.showinfo("Creación de Orden", "Orden creada exitosamente")
    entry_orden_cliente_id.delete(0, tk.END)
    actualizar_lista_ordenes(id_cliente)

def actualizar_lista_ordenes(id_cliente):
    listbox_ordenes.delete(0, tk.END)
    ordenes = tienda.obtener_ordenes_cliente(id_cliente)
    for orden in ordenes:
        listbox_ordenes.insert(tk.END, orden.mostrar_info())

def mostrar_detalles_orden(event):
    seleccion = listbox_ordenes.curselection()
    if seleccion:
        indice = seleccion[0]
        id_cliente = int(entry_orden_cliente_id.get())
        ordenes = tienda.obtener_ordenes_cliente(id_cliente)
        orden = ordenes[indice]
        messagebox.showinfo("Detalles de la Orden", orden.mostrar_info())

def agregar_item_a_orden():
    id_cliente = int(entry_orden_cliente_id.get())
    cliente = next((c for c in tienda.clientes if c.id_cliente == id_cliente), None)
    if cliente is None:
        messagebox.showerror("Error", "Cliente no encontrado")
        return
    producto_nombre = entry_item_producto.get()
    cantidad = int(entry_item_cantidad.get())
    producto = next((p for p in tienda.productos if p.nombre == producto_nombre), None)
    if producto is None:
        messagebox.showerror("Error", "Producto no encontrado")
        return
    orden = next((o for o in tienda.ordenes if o.cliente.id_cliente == id_cliente), None)
    if orden is None:
        messagebox.showerror("Error", "Orden no encontrada")
        return
    item = ItemOrden(producto, cantidad)
    orden.agregar_item(item)
    messagebox.showinfo("Item Agregado", f"Agregado {producto_nombre} x{cantidad} a la orden de {cliente.nombre} {cliente.apellido}")
    label_total_orden.config(text=f"Total de la Orden: ${orden.calcular_total():.2f}")
    entry_item_producto.delete(0, tk.END)
    entry_item_cantidad.delete(0, tk.END)
    actualizar_lista_ordenes(id_cliente)

# Configuración de la ventana principal
tienda = Tienda()

root = tk.Tk()
root.title("Sistema de Gestión de Tienda")
root.geometry("800x600")
root.configure(bg="lightgray")

notebook = ttk.Notebook(root)
notebook.pack(pady=10, expand=True)

frame_productos = tk.Frame(notebook, bg="lightblue")
frame_clientes = tk.Frame(notebook, bg="lightgreen")
frame_ordenes = tk.Frame(notebook, bg="lightyellow")

notebook.add(frame_productos, text="Productos")
notebook.add(frame_clientes, text="Clientes")
notebook.add(frame_ordenes, text="Órdenes")

# Frame Productos
tk.Label(frame_productos, text="Registro de Productos", bg="lightblue", font=("Arial", 16)).pack(pady=10)
tk.Label(frame_productos, text="Nombre", bg="lightblue", font=("Arial", 12)).pack()
entry_producto_nombre = tk.Entry(frame_productos, font=("Arial", 12))
entry_producto_nombre.pack()

tk.Label(frame_productos, text="Precio", bg="lightblue", font=("Arial", 12)).pack()
entry_producto_precio = tk.Entry(frame_productos, font=("Arial", 12))
entry_producto_precio.pack()

tk.Label(frame_productos, text="Categoría", bg="lightblue", font=("Arial", 12)).pack()
entry_producto_categoria = tk.Entry(frame_productos, font=("Arial", 12))
entry_producto_categoria.pack()

btn_registrar_producto = tk.Button(frame_productos, text="Registrar Producto", command=registrar_producto, font=("Arial", 12), bg="green", fg="white")
btn_registrar_producto.pack(pady=10)

# Frame Clientes
tk.Label(frame_clientes, text="Registro de Clientes", bg="lightgreen", font=("Arial", 16)).pack(pady=10)
tk.Label(frame_clientes, text="Nombre", bg="lightgreen", font=("Arial", 12)).pack()
entry_cliente_nombre = tk.Entry(frame_clientes, font=("Arial", 12))
entry_cliente_nombre.pack()

tk.Label(frame_clientes, text="Apellido", bg="lightgreen", font=("Arial", 12)).pack()
entry_cliente_apellido = tk.Entry(frame_clientes, font=("Arial", 12))
entry_cliente_apellido.pack()

tk.Label(frame_clientes, text="ID Cliente", bg="lightgreen", font=("Arial", 12)).pack()
entry_cliente_id = tk.Entry(frame_clientes, font=("Arial", 12))
entry_cliente_id.pack()

btn_registrar_cliente = tk.Button(frame_clientes, text="Registrar Cliente", command=registrar_cliente, font=("Arial", 12), bg="green", fg="white")
btn_registrar_cliente.pack(pady=10)

# Frame Órdenes
tk.Label(frame_ordenes, text="Crear Orden", bg="lightyellow", font=("Arial", 16)).pack(pady=10)
tk.Label(frame_ordenes, text="ID Cliente", bg="lightyellow", font=("Arial", 12)).pack()
entry_orden_cliente_id = tk.Entry(frame_ordenes, font=("Arial", 12))
entry_orden_cliente_id.pack()

btn_crear_orden = tk.Button(frame_ordenes, text="Crear Orden", command=crear_orden, font=("Arial", 12), bg="green", fg="white")
btn_crear_orden.pack(pady=10)

tk.Label(frame_ordenes, text="Agregar Item a la Orden", bg="lightyellow", font=("Arial", 16)).pack(pady=10)
tk.Label(frame_ordenes, text="Nombre del Producto", bg="lightyellow", font=("Arial", 12)).pack()
entry_item_producto = tk.Entry(frame_ordenes, font=("Arial", 12))
entry_item_producto.pack()

tk.Label(frame_ordenes, text="Cantidad", bg="lightyellow", font=("Arial", 12)).pack()
entry_item_cantidad = tk.Entry(frame_ordenes, font=("Arial", 12))
entry_item_cantidad.pack()

btn_agregar_item = tk.Button(frame_ordenes, text="Agregar Item", command=agregar_item_a_orden, font=("Arial", 12), bg="green", fg="white")
btn_agregar_item.pack(pady=10)

label_total_orden = tk.Label(frame_ordenes, text="Total de la Orden: $0.00", bg="lightyellow", font=("Arial", 12))
label_total_orden.pack(pady=10)

tk.Label(frame_ordenes, text="Órdenes del Cliente", bg="lightyellow", font=("Arial", 14)).pack(pady=10)
listbox_ordenes = tk.Listbox(frame_ordenes, font=("Arial", 12), width=50, height=10)
listbox_ordenes.pack(pady=10)

listbox_ordenes.bind('<<ListboxSelect>>', mostrar_detalles_orden)

root.mainloop()


Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_42060\3146838647.py", line 147, in agregar_item_a_orden
    id_cliente = int(entry_orden_cliente_id.get())
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: ''
Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\danie\AppData\Local\Programs\Python\Python312\Lib\tkinter\__init__.py", line 1967, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\danie\AppData\Local\Temp\ipykernel_42060\3146838647.py", line 147, in agregar_item_a_orden
    id_cliente = int(entry_orden_cliente_id.get())
                 ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
ValueError: invalid literal for int() with base 10: ''
Exce